In [2]:
import pyspark
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# import relevant parts of sklearn
from sklearn import cluster, datasets
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from scipy.cluster import hierarchy

%matplotlib inline
plt.style.use('seaborn-whitegrid')

import seaborn as sns
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.worker.memory", "8g").appName("Python Spark SQL basic example").getOrCreate()

# Loading Twitter Data

In [48]:
twitter = pd.read_parquet("sample.parquet", engine='pyarrow', columns=None)

In [8]:
pd.set_option('display.max_rows', 900)
pd.set_option('display.max_columns', 900)
pd.set_option('display.width', 1000)
twitter.head(7)

,create_at,favorite_count,geo_tag_city_id,geo_tag_city_name,geo_tag_county_id,geo_tag_county_name,geo_tag_state_id,geo_tag_state_name,id,in_reply_to_status,in_reply_to_user,is_retweet,lang,place_country,place_country_code,place_full_name,place_id,place_name,place_place_type,retweet_count,text,user_create_at,user_description,user_followers_count,user_friends_count,user_id,user_lang,user_location,user_name,user_profile_image_url,user_screen_name,user_statues_count
0,2016-01-01T00:00:27.000Z,0,1230000.0,Hialeah,12086,Miami-Dade,12,Florida,682712957118185473,-1,-1,False,en,United States,United States,"Hialeah, FL",629f4a26fed69cd3,Hialeah,city,0,Tryna get it in real quick before the turn up....,2010-10-08,Music Producer...Engineer....Winner..Productio...,5205,1745,200013652,en,Where ever the wave at.,Chize.,http://pbs.twimg.com/profile_images/6681745180...,ChizeOfficial,6181
1,2016-01-01T00:02:31.000Z,0,620018.0,Dublin,6001,Alameda,6,California,682713477715120128,-1,-1,False,en,United States,United States,"Dublin, CA",003253f0107acd32,Dublin,city,0,I'm running off on the plug right now https://...,2009-06-03,null,1942,1266,44515193,en,229✊- 478- 850,Brute Black Boobie,http://pbs.twimg.com/profile_images/6782158858...,BuddyNoLove,145115
2,2016-01-01T00:03:24.000Z,0,5148952.0,Manassas,51683,Manassas,51,Virginia,682713698272677888,682713447738568705,71644485,False,en,United States,United States,"Manassas, VA",1f703799b3940c20,Manassas,city,0,@Twittles23 I plan on running around Katsucon ...,2008-06-09,"Hentai NSX, lifting, food, cats, waifus.",49,55,15066662,en,Virginia,cosplayerkyo,http://pbs.twimg.com/profile_images/6772800631...,cosplayerkyo,1621
3,2016-01-01T00:06:42.000Z,0,1250575.0,Oakland Park,12011,Broward,12,Florida,682714528413540352,682713382223507457,1853895816,False,en,United States,United States,"Oakland Park, FL",b230af2256484995,Oakland Park,city,0,@KittyKnits Could be worse. I'm waiting for th...,2009-06-27,null,1625,2039,51540429,en,"Ft. Lauderdale, Florida USA",Ray Lauderback,http://pbs.twimg.com/profile_images/5936642848...,okfor1,20210
4,2016-01-01T00:09:10.000Z,0,5351300.0,Olympia,53067,Thurston,53,Washington,682715151015907328,-1,-1,False,en,United States,United States,"Olympia, WA",375aa1ab4abd79b1,Olympia,city,0,Wishing you a happy and healthy 2016! See you ...,2012-10-06,"The Capital City Marathon, Half Marathon, 5 Mi...",151,251,865405321,en,"Olympia, WA",CapitalCityMarathon,http://pbs.twimg.com/profile_images/5226124486...,CapCityMarathon,336
5,2016-01-01T00:09:51.000Z,0,1304000.0,Atlanta,13121,Fulton,13,Georgia,682715319975211008,-1,-1,False,en,United States,United States,"Atlanta, GA",8173485c72e78ca5,Atlanta,city,0,Last person in the gym for the year! 💁🏻 @ L A ...,2010-10-25,take me to the gym?,30,58,207546325,en,"Atlanta, GA",Morgan Brown,http://pbs.twimg.com/profile_images/6824350692...,m0rgancar0l,25
6,2016-01-01T00:10:00.000Z,0,1208150.0,Brandon,12057,Hillsborough,12,Florida,682715359535824896,-1,-1,False,en,United States,United States,"Florida, USA",4ec01c9dbc693497,Florida,admin,0,"#sanctuary (@ Fitness for $10 in Brandon, FL) ...",2009-03-24,Just a small town girl. Living in a lonely world.,162,282,26328877,en,"Tampa, FL",Sonny,http://pbs.twimg.com/profile_images/6673098123...,SonnyMendoza,737


# Loading Population Data

In [13]:
population=pd.read_csv("pouplation3.csv")
population.head(10)

,California Cities by Population Rank,City,Population
0,1,Los Angeles,"3,949,776"
1,2,San Diego,"1,390,966"
2,3,San Jose,"1,023,031"
3,4,San Francisco,"864,263"
4,5,Fresno,"519,037"
5,6,Sacramento,"489,650"
6,7,Long Beach,"470,489"
7,8,Oakland,"417,442"
8,9,Bakersfield,"372,680"
9,10,Anaheim,"349,007"


# Loading Income Data 

In [21]:
income=pd.read_csv("income.csv")
income.head(10)

,PLACE,COUNTY,POPULATION,POPULATION DENSITY,PER CAPITA INCOME,MEDIAN HOUSEHOLD INCOME,MEDIAN FAMILY INCOME
0,Los Angeles,Los Angeles,"3,862,210","8,240.80","$28,320","$49,682","$54,171"
1,San Diego,San Diego,"1,341,510","4,125.30","$33,789","$65,753","$78,414"
2,San Jose,Santa Clara,"986,320","5,586.00","$34,992","$83,787","$92,379"
3,San Francisco,San Francisco,"829,072","17,680.10","$49,986","$78,378","$93,391"
4,Fresno,Fresno,"506,132","4,470.20","$19,226","$41,455","$45,554"
5,Sacramento,Sacramento,"476,075","4,861.70","$26,060","$50,013","$57,200"
6,Long Beach,Los Angeles,"468,594","9,315.80","$27,014","$52,944","$59,710"
7,Oakland,Alameda,"402,339","7,198.40","$32,566","$52,962","$60,472"
8,Bakersfield,Kern,"358,700","2,414.70","$23,786","$56,842","$61,994"
9,Anaheim,Orange,"342,973","6,862.60","$23,990","$59,707","$63,281"


# Loading Area Data

In [22]:
city_county=pd.read_csv("citycounty3.csv")
city_county.head(10)

,Name,Type,County,Area
0,Adelanto,City,San Bernardino,145.1
1,Agoura Hills,City,Los Angeles,20.2
2,Alameda,City,Alameda,27.5
3,Albany,City,Alameda,4.6
4,Alhambra,City,Los Angeles,19.8
5,Aliso Viejo,City,Orange,19.3
6,Alturas,City,Modoc,6.3
7,Amador City,City,Amador,0.8
8,American Canyon,City,Napa,12.5
9,Anaheim,City,Orange,129.1


# Loading Air Quality Data

In [23]:
AQI=pd.read_csv("export_AQI2.csv")
AQI.head(10)

,DATA,PLACE,COUNTY,STATE,AQI_VALUE
0,2016-01-01,Livermore,Alameda,California,41
1,2016-01-02,Livermore,Alameda,California,53
2,2016-01-03,Livermore,Alameda,California,72
3,2016-01-04,Livermore,Alameda,California,61
4,2016-01-05,Livermore,Alameda,California,21
5,2016-01-06,Livermore,Alameda,California,17
6,2016-01-07,Livermore,Alameda,California,40
7,2016-01-08,Livermore,Alameda,California,44
8,2016-01-09,Livermore,Alameda,California,40
9,2016-01-10,Livermore,Alameda,California,44


# Loading Gender Data

In [24]:
gender=pd.read_csv("gender.csv")
gender.head(10)

,fips,county,year,age,pop_female,pop_male,pop_total
0,6001,ALAMEDA,1970,0,8533,8671,17204
1,6001,ALAMEDA,1970,1,8151,8252,16403
2,6001,ALAMEDA,1970,2,7753,8015,15768
3,6001,ALAMEDA,1970,3,8018,8412,16430
4,6001,ALAMEDA,1970,4,8551,8648,17199
5,6001,ALAMEDA,1970,5,9012,9208,18220
6,6001,ALAMEDA,1970,6,9637,9636,19273
7,6001,ALAMEDA,1970,7,9561,9713,19274
8,6001,ALAMEDA,1970,8,9648,10008,19656
9,6001,ALAMEDA,1970,9,9718,10093,19811


# Loading Walk/Bike Scores Data

In [142]:
exercise=pd.read_csv("exercise score 1.csv")
exercise.drop(columns=['Unnamed: 4']).head(10)

,City,Zip Code,Walk Score,Bike Score
0,West Hollywood,NaN,91,54
1,San Francisco,94112.0,86,71
2,Hermosa Beach,90254.0,84,46
3,Santa Monica,90405.0,83,84
4,Berkeley,94704.0,81,85
5,Albany,94706.0,80,46
6,Huntington Park,NaN,80,58
7,Beverly Hills,90212.0,78,54
8,Maywood,90270.0,77,63
9,Lawndale,NaN,76,59


# Loading Holiday Data

In [26]:
workday=pd.read_csv("HOLIDAY.csv")
workday.head(10)

,DATE,WORKDAY
0,2016-01-01,0.0
1,2016-01-02,0.0
2,2016-01-03,0.0
3,2016-01-04,1.0
4,2016-01-05,1.0
5,2016-01-06,1.0
6,2016-01-07,1.0
7,2016-01-08,1.0
8,2016-01-09,0.0
9,2016-01-10,0.0


# Loading Weather Data

In [149]:
weather=pd.read_csv("export_weather2.csv")
weather=weather.drop(columns=['AWND','TAVG','Unnamed: 8','Unnamed: 9','WDF2'])
weather.loc[weather['PLACE'] == 'Los Angeles'].tail(10)

,PLACE,DATE,ELEVATION,PRCP,TMAX
1032564,Los Angeles,2018-04-21,54.6,0.0,83.0
1032565,Los Angeles,2018-04-22,54.6,0.0,79.0
1032566,Los Angeles,2018-04-23,54.6,0.0,76.0
1032567,Los Angeles,2018-04-24,54.6,0.0,70.0
1032568,Los Angeles,2018-04-25,54.6,0.0,70.0
1032569,Los Angeles,2018-04-26,54.6,0.0,71.0
1032570,Los Angeles,2018-04-27,54.6,0.0,70.0
1032571,Los Angeles,2018-04-28,54.6,0.0,70.0
1032572,Los Angeles,2018-04-29,54.6,0.0,69.0
1032573,Los Angeles,2018-04-30,54.6,0.0,68.0


# Create New DataFrame to Merge ALL Database

In [34]:
#twitter['text']=twitter['text'].str.split(' ')

In [35]:
#Counts Key Words: 

In [49]:
counts=twitter['text'].str.count("running")+twitter['text'].str.count("asthma")+twitter['text'].str.count("outdoor")+twitter['text'].str.count("activity")+twitter['text'].str.count("activities")+twitter['text'].str.count("hiking")+twitter['text'].str.count("jogging")+twitter['text'].str.count("climbing")+twitter['text'].str.count("camping")+twitter['text'].str.count("fishing")+twitter['text'].str.count("hunting")+twitter['text'].str.count("backpacking")+twitter['text'].str.count("biking")+twitter['text'].str.count("Air")+twitter['text'].str.count("air")+twitter['text'].str.count("Pollution")+twitter['text'].str.count("pollution")+twitter['text'].str.count("Running")+twitter['text'].str.count("Asthma")+twitter['text'].str.count("Outdoor")+twitter['text'].str.count("Activity")+twitter['text'].str.count("Activities")+twitter['text'].str.count("Hiking")+twitter['text'].str.count("Jogging")+twitter['text'].str.count("Climbing")+twitter['text'].str.count("Camping")+twitter['text'].str.count("Fishing")+twitter['text'].str.count("Hunting")+twitter['text'].str.count("Backpacking")+twitter['text'].str.count("Biking")

# Count Twitter with keywords and create new dataframe

In [50]:
merge_data=pd.DataFrame({'create':twitter.create_at,'geo_tag_city_name':twitter.geo_tag_city_name,'count': counts})

In [42]:
#Split 'create' Cretae to get date: 

In [51]:
merge_data['create']=merge_data['create'].str.split('T').str[0]

In [53]:
merge_data.head(20)

,create,geo_tag_city_name,count
514550,2018-05-06,Aaronsburg,1
298267,2017-03-27,Aasco,1
5230265,2016-01-02,Abbeville,1
655547,2016-01-03,Abbeville,1
5231347,2016-01-03,Abbeville,1
5884008,2016-01-03,Abbeville,1
1964501,2016-01-08,Abbeville,1
3275172,2016-01-12,Abbeville,1
4582405,2016-01-12,Abbeville,1
5237087,2016-01-12,Abbeville,1


In [54]:
merge_data=merge_data.groupby(['create','geo_tag_city_name'])['count'].sum().reset_index(name="counts")

In [55]:
merge_data.count()

create               2038429
geo_tag_city_name    2038429
counts               2038429
dtype: int64

In [56]:
pouplation=pouplation.drop(columns=['California Cities by Population Rank'])

In [57]:
merge_data_1= merge_data.merge(pouplation,left_on='geo_tag_city_name', right_on='City')

In [58]:
merge_data_1=merge_data_1.drop(columns=['City'])

In [59]:
merge_data_1.head(5)

,create,geo_tag_city_name,counts,Population
0,2016-01-01,Alameda,2,"78,246"
1,2016-01-02,Alameda,31,"78,246"
2,2016-01-03,Alameda,22,"78,246"
3,2016-01-05,Alameda,27,"78,246"
4,2016-01-06,Alameda,19,"78,246"


# Merge Income Dataset According to key['geo_tag_city_name']

In [60]:
merge_data_2 = merge_data_1.merge(income,left_on='geo_tag_city_name', right_on='PLACE')

In [62]:
merge_data_2 .head(5)

,create,geo_tag_city_name,counts,Population,PLACE,COUNTY,POPULATION,POPULATION DENSITY,PER CAPITA INCOME,MEDIAN HOUSEHOLD INCOME,MEDIAN FAMILY INCOME
0,2016-01-01,Alameda,2,"78,246",Alameda,Alameda,"75,763","7,255.60","$42,331","$76,439","$93,427"
1,2016-01-02,Alameda,31,"78,246",Alameda,Alameda,"75,763","7,255.60","$42,331","$76,439","$93,427"
2,2016-01-03,Alameda,22,"78,246",Alameda,Alameda,"75,763","7,255.60","$42,331","$76,439","$93,427"
3,2016-01-05,Alameda,27,"78,246",Alameda,Alameda,"75,763","7,255.60","$42,331","$76,439","$93,427"
4,2016-01-06,Alameda,19,"78,246",Alameda,Alameda,"75,763","7,255.60","$42,331","$76,439","$93,427"


In [63]:
merge_data_3=merge_data_2.drop(columns=['PLACE','COUNTY','POPULATION','POPULATION DENSITY','PER CAPITA INCOME','MEDIAN HOUSEHOLD INCOME'])

In [64]:
merge_data_3.head(1)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME
0,2016-01-01,Alameda,2,"78,246","$93,427"


# Merge County Dataset According to key['geo_tag_city_name']

In [65]:
merge_data_4 = merge_data_3.merge(city_county,left_on='geo_tag_city_name', right_on='Name')

In [66]:
merge_data_4.head(5)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,Name,Type,County,Area
0,2016-01-01,Alameda,2,"78,246","$93,427",Alameda,City,Alameda,27.5
1,2016-01-02,Alameda,31,"78,246","$93,427",Alameda,City,Alameda,27.5
2,2016-01-03,Alameda,22,"78,246","$93,427",Alameda,City,Alameda,27.5
3,2016-01-05,Alameda,27,"78,246","$93,427",Alameda,City,Alameda,27.5
4,2016-01-06,Alameda,19,"78,246","$93,427",Alameda,City,Alameda,27.5


In [67]:
merge_data_5=merge_data_4.drop(columns=['Name','Type'])

In [68]:
merge_data_5.head(3)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County,Area
0,2016-01-01,Alameda,2,"78,246","$93,427",Alameda,27.5
1,2016-01-02,Alameda,31,"78,246","$93,427",Alameda,27.5
2,2016-01-03,Alameda,22,"78,246","$93,427",Alameda,27.5


In [76]:
gender2 = gender.groupby('county')
gender2=gender2.apply(lambda x: x[x['age'] <50]['pop_total'].sum())/gender2.apply(lambda x: x['pop_total'].sum())
gender2 = pd.DataFrame(gender2).reset_index()
gender2.columns = ['county', 'percentage']
gender2.head(5)

,county,percentage
0,ALAMEDA,0.676553
1,ALPINE,0.609305
2,AMADOR,0.557627
3,BUTTE,0.671242
4,CALAVERAS,0.530425


# Merge Age<50 Dataset According to key['County']

In [77]:
merge_data_5['County']=merge_data_5['County'].str.upper()

In [80]:
merge_data_6 = merge_data_5.merge(gender2,left_on='County', right_on='county')

In [79]:
merge_data_6=merge_data_6.drop(columns=['county'])

In [82]:
merge_data_6.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County,Area,county,percentage
0,2016-01-01,Alameda,2,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
1,2016-01-02,Alameda,31,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
2,2016-01-03,Alameda,22,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
3,2016-01-05,Alameda,27,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
4,2016-01-06,Alameda,19,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
5,2016-01-07,Alameda,21,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
6,2016-01-08,Alameda,31,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
7,2016-01-09,Alameda,40,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
8,2016-01-10,Alameda,20,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553
9,2016-01-11,Alameda,1,"78,246","$93,427",ALAMEDA,27.5,ALAMEDA,0.676553


In [83]:
merge_data_6['MEDIAN FAMILY INCOME']=merge_data_6['MEDIAN FAMILY INCOME'].str.lstrip('$') 
#Make them into numbers instead of string

# Merge exercise Dataset According to key['geo_tag_city_name']

In [86]:
exercise.iloc[32, exercise.columns.get_loc('City')] = 'Los Angeles'

In [88]:
merge_data_7 = merge_data_6.merge(exercise,left_on='geo_tag_city_name', right_on='City')

In [89]:
merge_data_7.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County,Area,county,percentage,City,Zip Code,Walk Score,Bike Score,Unnamed: 4
0,2016-01-01,Alameda,2,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
1,2016-01-02,Alameda,31,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
2,2016-01-03,Alameda,22,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
3,2016-01-05,Alameda,27,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
4,2016-01-06,Alameda,19,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
5,2016-01-07,Alameda,21,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
6,2016-01-08,Alameda,31,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
7,2016-01-09,Alameda,40,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
8,2016-01-10,Alameda,20,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN
9,2016-01-11,Alameda,1,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,Alameda,94501.0,65,73,NaN


In [90]:
merge_data_7=merge_data_7.drop(columns=['City','Zip Code','Unnamed: 4'])

In [92]:
merge_data_7.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County,Area,county,percentage,Walk Score,Bike Score
0,2016-01-01,Alameda,2,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
1,2016-01-02,Alameda,31,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
2,2016-01-03,Alameda,22,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
3,2016-01-05,Alameda,27,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
4,2016-01-06,Alameda,19,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
5,2016-01-07,Alameda,21,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
6,2016-01-08,Alameda,31,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
7,2016-01-09,Alameda,40,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
8,2016-01-10,Alameda,20,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73
9,2016-01-11,Alameda,1,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73


# Merge Holiday Dataset According to key['create'']

In [93]:
merge_data_8 = merge_data_7.merge(workday,left_on='create', right_on='DATE')

In [94]:
merge_data_8 =merge_data_8 .drop(columns=['DATE'])

In [95]:
merge_data_8 .head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County,Area,county,percentage,Walk Score,Bike Score,WORKDAY
0,2016-01-01,Alameda,2,"78,246","93,427",ALAMEDA,27.5,ALAMEDA,0.676553,65,73,0.0
1,2016-01-01,Berkeley,6,"120,179","114,720",ALAMEDA,27.1,ALAMEDA,0.676553,81,85,0.0
2,2016-01-01,Dublin,4,"57,022","128,737",ALAMEDA,38.6,ALAMEDA,0.676553,34,56,0.0
3,2016-01-01,Fremont,1,"230,964","112,347",ALAMEDA,200.6,ALAMEDA,0.676553,44,49,0.0
4,2016-01-01,Hayward,1,"156,917","69,415",ALAMEDA,117.4,ALAMEDA,0.676553,53,52,0.0
5,2016-01-01,Livermore,1,"88,232","111,950",ALAMEDA,65.2,ALAMEDA,0.676553,37,64,0.0
6,2016-01-01,Newark,4,"45,554","91,103",ALAMEDA,35.9,ALAMEDA,0.676553,47,45,0.0
7,2016-01-01,Oakland,8,"417,442","60,472",ALAMEDA,144.5,ALAMEDA,0.676553,72,65,0.0
8,2016-01-01,San Leandro,1,"89,910","76,781",ALAMEDA,34.6,ALAMEDA,0.676553,62,55,0.0
9,2016-01-01,Union City,2,"74,354","93,050",ALAMEDA,50.4,ALAMEDA,0.676553,44,49,0.0


In [96]:
merge_data_8 ['MEDIAN FAMILY INCOME']=merge_data_8 ['MEDIAN FAMILY INCOME'].str.lstrip('$')

In [97]:
merge_data_8 ['MEDIAN FAMILY INCOME']=merge_data_8 ['MEDIAN FAMILY INCOME'].str.replace(",","")

In [98]:
merge_data_8 ['Population']=merge_data_8 ['Population'].str.replace(",","")

In [99]:
merge_data_8 .head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County,Area,county,percentage,Walk Score,Bike Score,WORKDAY
0,2016-01-01,Alameda,2,78246,93427,ALAMEDA,27.5,ALAMEDA,0.676553,65,73,0.0
1,2016-01-01,Berkeley,6,120179,114720,ALAMEDA,27.1,ALAMEDA,0.676553,81,85,0.0
2,2016-01-01,Dublin,4,57022,128737,ALAMEDA,38.6,ALAMEDA,0.676553,34,56,0.0
3,2016-01-01,Fremont,1,230964,112347,ALAMEDA,200.6,ALAMEDA,0.676553,44,49,0.0
4,2016-01-01,Hayward,1,156917,69415,ALAMEDA,117.4,ALAMEDA,0.676553,53,52,0.0
5,2016-01-01,Livermore,1,88232,111950,ALAMEDA,65.2,ALAMEDA,0.676553,37,64,0.0
6,2016-01-01,Newark,4,45554,91103,ALAMEDA,35.9,ALAMEDA,0.676553,47,45,0.0
7,2016-01-01,Oakland,8,417442,60472,ALAMEDA,144.5,ALAMEDA,0.676553,72,65,0.0
8,2016-01-01,San Leandro,1,89910,76781,ALAMEDA,34.6,ALAMEDA,0.676553,62,55,0.0
9,2016-01-01,Union City,2,74354,93050,ALAMEDA,50.4,ALAMEDA,0.676553,44,49,0.0


In [100]:
merge_data_9 =merge_data_8 .drop(columns=['Area'])

# Merge City_County Dataset According to key['geo_tag_city_name']

In [101]:
city_county.iloc[121, city_county.columns.get_loc('Area')] = 34

In [102]:
merge_data_10 = merge_data_9.merge(city_county,left_on='geo_tag_city_name', right_on='Name')

In [103]:
merge_data_10=merge_data_10.drop(columns=['Name','Type','County_y'])

In [104]:
merge_data_10['Area']=merge_data_10['Area'].str.replace(",","")

In [105]:
merge_data_10.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County_x,county,percentage,Walk Score,Bike Score,WORKDAY,Area
0,2016-01-01,Alameda,2,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5
1,2016-01-02,Alameda,31,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5
2,2016-01-03,Alameda,22,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5
3,2016-01-05,Alameda,27,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5
4,2016-01-06,Alameda,19,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5
5,2016-01-07,Alameda,21,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5
6,2016-01-08,Alameda,31,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5
7,2016-01-09,Alameda,40,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5
8,2016-01-10,Alameda,20,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5
9,2016-01-11,Alameda,1,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5


# Make all numbers to numeric

In [106]:
merge_data_10['MEDIAN FAMILY INCOME']=pd.to_numeric(merge_data_10['MEDIAN FAMILY INCOME'])
merge_data_10['Population']=pd.to_numeric(merge_data_10['Population'])
merge_data_10['Area']=pd.to_numeric(merge_data_10['Area'])

# Merge AQI Dataset According to key['create','geo_tag_city_name']

In [107]:
AQI.columns = ['create', 'geo_tag_city_name','County_x','STATE','AQI_VALUE']

In [108]:
AQI['County_x']=AQI['County_x'].str.upper()

In [115]:
merge_data_11=merge_data_10.merge(AQI,on=['create','geo_tag_city_name'],how='left')

In [116]:
merge_data_11.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,County_x_x,county,percentage,Walk Score,Bike Score,WORKDAY,Area,County_x_y,STATE,AQI_VALUE
0,2016-01-01,Alameda,2,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN
1,2016-01-02,Alameda,31,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN
2,2016-01-03,Alameda,22,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN
3,2016-01-05,Alameda,27,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN
4,2016-01-06,Alameda,19,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN
5,2016-01-07,Alameda,21,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN
6,2016-01-08,Alameda,31,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN
7,2016-01-09,Alameda,40,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN
8,2016-01-10,Alameda,20,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN
9,2016-01-11,Alameda,1,78246,93427,ALAMEDA,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN


In [119]:
merge_data_11=merge_data_11.drop(columns=['STATE','County_x_y','County_x_x'])

In [120]:
merge_data_11.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,county,percentage,Walk Score,Bike Score,WORKDAY,Area,AQI_VALUE
0,2016-01-01,Alameda,2,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN
1,2016-01-02,Alameda,31,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN
2,2016-01-03,Alameda,22,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN
3,2016-01-05,Alameda,27,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN
4,2016-01-06,Alameda,19,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN
5,2016-01-07,Alameda,21,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN
6,2016-01-08,Alameda,31,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN
7,2016-01-09,Alameda,40,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN
8,2016-01-10,Alameda,20,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN
9,2016-01-11,Alameda,1,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN


In [113]:
weather=weather.drop(columns=['AWND','TAVG','WDF2','Unnamed: 8','Unnamed: 9'])

In [114]:
weather.head(10)

,PLACE,DATE,ELEVATION,PRCP,TMAX
0,Piedmont,2016-01-01,113.4,0.00,NaN
1,Piedmont,2016-01-02,113.4,0.00,NaN
2,Piedmont,2016-01-03,113.4,0.00,NaN
3,Piedmont,2016-01-04,113.4,0.05,NaN
4,Piedmont,2016-01-05,113.4,0.72,NaN
5,Piedmont,2016-01-06,113.4,0.94,NaN
6,Piedmont,2016-01-07,113.4,0.60,NaN
7,Piedmont,2016-01-08,113.4,0.00,NaN
8,Piedmont,2016-01-09,113.4,0.09,NaN
9,Piedmont,2016-01-10,113.4,0.00,NaN


In [121]:
weather.columns = ['geo_tag_city_name','create','ELEVATION','PRCP','TMAX']

# Merge Weather Dataset According to key['create','geo_tag_city_name']

In [122]:
merge_data_12=merge_data_11.merge(weather,on=['create','geo_tag_city_name'],how='left')

In [124]:
merge_data_12.count()

create                  2254859
geo_tag_city_name       2254859
counts                  2254859
Population              2254859
MEDIAN FAMILY INCOME    2254859
county                  2254859
percentage              2254859
Walk Score              2254859
Bike Score              2254859
WORKDAY                 2254859
Area                    2254105
AQI_VALUE               1528823
ELEVATION_x             2126153
PRCP_x                  1979785
TMAX_x                   665410
ELEVATION_y             2126153
PRCP_y                  1979785
TMAX_y                   665410
dtype: int64

In [128]:
merge_data_13=merge_data_13.drop(columns=['ELEVATION_y','PRCP_y','TMAX_y'])

# Add Season Factor

In [130]:
merge_data_13['season']=None

In [132]:
merge_data_13.loc[merge_data_13['create'].str.contains('-12-'),'season'] = 'winter'
merge_data_13.loc[merge_data_13['create'].str.contains('-01-'),'season'] = 'winter'
merge_data_13.loc[merge_data_13['create'].str.contains('-02-'),'season'] = 'winter'
merge_data_13.loc[merge_data_13['create'].str.contains('-03-'),'season'] = 'spring'
merge_data_13.loc[merge_data_13['create'].str.contains('-04-'),'season'] = 'spring'
merge_data_13.loc[merge_data_13['create'].str.contains('-05-'),'season'] = 'spring'
merge_data_13.loc[merge_data_13['create'].str.contains('-06-'),'season'] = 'summer'
merge_data_13.loc[merge_data_13['create'].str.contains('-07-'),'season'] = 'summer'
merge_data_13.loc[merge_data_13['create'].str.contains('-08-'),'season'] = 'summer'
merge_data_13.loc[merge_data_13['create'].str.contains('-09-'),'season'] = 'fall'
merge_data_13.loc[merge_data_13['create'].str.contains('-10-'),'season'] = 'fall'
merge_data_13.loc[merge_data_13['create'].str.contains('-11-'),'season'] = 'fall'

In [135]:
merge_data_13['create']=pd.to_datetime(merge_data_13['create'])

In [136]:
merge_data_13.head(10)

,create,geo_tag_city_name,counts,Population,MEDIAN FAMILY INCOME,county,percentage,Walk Score,Bike Score,WORKDAY,Area,AQI_VALUE,ELEVATION_x,PRCP_x,TMAX_x,season
0,2016-01-01,Alameda,2,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN,NaN,winter
1,2016-01-02,Alameda,31,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN,NaN,winter
2,2016-01-03,Alameda,22,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN,NaN,winter
3,2016-01-05,Alameda,27,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN,NaN,winter
4,2016-01-06,Alameda,19,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN,NaN,winter
5,2016-01-07,Alameda,21,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN,NaN,winter
6,2016-01-08,Alameda,31,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN,NaN,winter
7,2016-01-09,Alameda,40,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN,NaN,winter
8,2016-01-10,Alameda,20,78246,93427,ALAMEDA,0.676553,65,73,0.0,27.5,NaN,NaN,NaN,NaN,winter
9,2016-01-11,Alameda,1,78246,93427,ALAMEDA,0.676553,65,73,1.0,27.5,NaN,NaN,NaN,NaN,winter


In [138]:
fillna=merge_data_13.groupby('create').mean().reset_index()
fillna=fillna.drop(columns=['counts','Population','MEDIAN FAMILY INCOME','percentage','Walk Score','Bike Score','WORKDAY','Area'])
fillna['create']=pd.to_datetime(fillna['create'])
merge_data_13['create']=pd.to_datetime(merge_data_13['create'])

In [139]:
merge_data_14=merge_data_13.merge(fillna, left_on='create', right_on='create')
merge_data_14.AQI_VALUE_x.fillna(merge_data_14.AQI_VALUE_y, inplace=True)
merge_data_14.ELEVATION_x_x.fillna(merge_data_14.ELEVATION_x_y, inplace=True)
merge_data_14.PRCP_x_x.fillna(merge_data_14.PRCP_x_y, inplace=True)
merge_data_14.TMAX_x_x.fillna(merge_data_14.TMAX_x_y, inplace=True)

# Change the name of the variables

In [140]:
merge_data_14=merge_data_14.drop(columns=['AQI_VALUE_y','ELEVATION_x_y','PRCP_x_y','TMAX_x_y'])
merge_data_14=merge_data_14.dropna()
merge_data_14=merge_data_14.drop_duplicates(['create','geo_tag_city_name'])
merge_data_14.columns =['create',"geo_tag_city_name","counts",'Population',"INCOME",'county',"percentage","Walk_Score","Bike_Score","WORKDAY","Area","AQI_VALUE","ELEVATION","PRCP","TMAX","season"]
merge_data_14=merge_data_14.drop(columns=['county'])
merge_data_14['season'] = merge_data_14['season'].astype("category").cat.codes

# Normalize the population data

In [ ]:
merge_data_14['COUNT_BY_POPULATION'] = 100000*merge_data_14['counts']/merge_data_14['Population']

In [141]:
merge_data_14.head(10)

,create,geo_tag_city_name,counts,Population,INCOME,percentage,Walk_Score,Bike_Score,WORKDAY,Area,AQI_VALUE,ELEVATION,PRCP,TMAX,season,COUNT_BY_POPULATION
0,2016-01-01,Alameda,2,78246,93427,0.676553,65,73,0.0,27.5,50.271697,155.045081,0.0,57.356678,3,2.556041
1,2016-01-01,Berkeley,6,120179,114720,0.676553,81,85,0.0,27.1,50.271697,349.900000,0.0,57.356678,3,4.992553
5,2016-01-01,Dublin,4,57022,128737,0.676553,34,56,0.0,38.6,50.271697,155.045081,0.0,57.356678,3,7.014836
6,2016-01-01,Fremont,1,230964,112347,0.676553,44,49,0.0,200.6,50.271697,9.100000,0.0,57.356678,3,0.432968
15,2016-01-01,Hayward,1,156917,69415,0.676553,53,52,0.0,117.4,50.271697,155.045081,0.0,57.356678,3,0.637280
16,2016-01-01,Livermore,1,88232,111950,0.676553,37,64,0.0,65.2,41.000000,146.300000,0.0,46.000000,3,1.133376
20,2016-01-01,Newark,4,45554,91103,0.676553,47,45,0.0,35.9,50.271697,155.045081,0.0,57.356678,3,8.780788
21,2016-01-01,Oakland,8,417442,60472,0.676553,72,65,0.0,144.5,13.000000,120.400000,0.0,57.356678,3,1.916434
168,2016-01-01,San Leandro,1,89910,76781,0.676553,62,55,0.0,34.6,50.271697,155.045081,0.0,57.356678,3,1.112223
169,2016-01-01,Union City,2,74354,93050,0.676553,44,49,0.0,50.4,50.271697,155.045081,0.0,57.356678,3,2.689835


# Export the new full dataset

In [129]:
export_csv = merge_data_14.to_csv (r'fulltextdata_2.csv', index = None, header=True)